<a href="https://colab.research.google.com/github/grace0607/OAI-API-Testing/blob/main/OAI_API_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI API Testing
This notebook makes API calls to allow users to have multi-turn conversations with OpenAI models. Refer to https://platform.openai.com/docs/models for the full list of models and their specs.

You can click 'Run all' in the Runtime tab to get started or 'Cmd+F9' as a shortcut.

Last updated: 1/21/2025

# **1. Install packages and configure API key**
  Run this cell to install the required packages and to initialize the OpenAI client using your API key. Save your API key as an environment variable to the Colab "Secrets" ('Name' field should be a sort of nickname for your API key of your choice and 'Value' should be the actual value of your API key)and allow "Notebook access."
  
  You only need to run this cell once.

In [26]:
# Install required packages
!pip install openai

import openai
from datetime import datetime
import json
from google.colab import userdata

# Initialize the OpenAI client
client = openai.OpenAI(
    api_key=userdata.get('openai')
)

# **2. Create functions**
  Run this cell to define all the API models and functions we need to test the API. You shouldn't see anything happen if the run is successful. It's a silent success!

In [27]:
# Define available models with their specifications
MODELS = {
    # GPT-4o Series
    "gpt-4o": {"context_window": 128000, "max_output": 16384, "description": "Same as gpt-4o-2024-08-06"},
    "gpt-4o-2024-08-06": {"context_window": 128000, "max_output": 16384},
    "gpt-4o-2024-11-20": {"context_window": 128000, "max_output": 16384},
    "gpt-4o-2024-05-13": {"context_window": 128000, "max_output": 4096},
    "chatgpt-4o-latest": {"context_window": 128000, "max_output": 16384, "description": "GPT-4o used in ChatGPT UI"},

    # GPT-4o Mini Series
    "gpt-4o-mini-2024-07-18": {"context_window": 128000, "max_output": 16384},

    # o1 Series
    "o1-2024-12-17": {"context_window": 200000, "max_output": 100000, "description": "Same as o1"},
    "o1-mini-2024-09-12": {"context_window": 128000, "max_output": 65536},
    "o1-preview-2024-09-12": {"context_window": 128000, "max_output": 32768},

    # DALL-E Series
    "dall-e-3": {"description": "Latest DALL·E model released in Nov 2023"},
    "dall-e-2": {"description": "Previous DALL·E model with 4x greater resolution than original"},

    # GPT-4o Realtime Preview
    "gpt-4o-realtime-preview-2024-12-17": {"context_window": 128000, "max_output": 4096},
    "gpt-4o-realtime-preview-2024-10-01": {"context_window": 128000, "max_output": 4096},
    "gpt-4o-mini-realtime-preview-2024-12-17": {"context_window": 128000, "max_output": 4096},

    # GPT-4o Audio Preview
    "gpt-4o-audio-preview-2024-12-17": {"context_window": 128000, "max_output": 16384},
    "gpt-4o-audio-preview-2024-10-01": {"context_window": 128000, "max_output": 16384},
    "gpt-4o-mini-audio-preview-2024-12-17": {"context_window": 128000, "max_output": 16384},

    # GPT-4 Turbo and GPT-4
    "gpt-4-turbo-2024-04-09": {"context_window": 128000, "max_output": 4096, "description": "Same as gpt-4-turbo"},
    "gpt-4-0125-preview": {"context_window": 128000, "max_output": 4096},
    "gpt-4-1106-preview": {"context_window": 128000, "max_output": 4096},
    "gpt-4-0613": {"context_window": 8192, "max_output": 8192},
    "gpt-4-0314": {"context_window": 8192, "max_output": 8192},

    # GPT-3.5 Turbo
    "gpt-3.5-turbo-0125": {"context_window": 16385, "max_output": 4096, "description": "Same as gpt-3.5-turbo"},
    "gpt-3.5-turbo-1106": {"context_window": 16385, "max_output": 4096},
    "gpt-3.5-turbo-instruct": {"context_window": 4096, "max_output": 4096},
}

def initialize_chat(model="gpt-4o"):
    """Initialize a new chat session with specified model."""
    return {
        "model": model,
        "messages": [],
        "start_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

def send_message(chat_session, user_message):
    """Send a message to the API and return the response."""
    # Add user message to history
    chat_session["messages"].append({"role": "user", "content": user_message})

    try:
        # Send to API
        response = client.chat.completions.create(
            model=chat_session["model"],
            messages=chat_session["messages"],
            temperature=1.0
        )

        # Get assistant's response
        assistant_message = response.choices[0].message.content

        # Add to history
        chat_session["messages"].append({"role": "assistant", "content": assistant_message})

        return assistant_message

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def print_transcript(chat_session):
    """Print the full conversation transcript."""
    print(f"\nChat Transcript - Started at {chat_session['start_time']}")
    print(f"Model: {chat_session['model']}")
    model_info = MODELS[chat_session['model']]
    if 'context_window' in model_info:
        print(f"Context Window: {model_info['context_window']} tokens")
        print(f"Max Output: {model_info['max_output']} tokens")
    if 'description' in model_info:
        print(f"Description: {model_info['description']}")
    print("\n" + "-" * 80)

    for msg in chat_session["messages"]:
        role = msg["role"].upper()
        content = msg["content"]
        print(f"\n{role}:")
        print(content)
        print("-" * 80)

def export_transcript(chat_session, filename=None):
    """Export the conversation transcript to a JSON file."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"chat_transcript_{timestamp}.json"

    transcript_data = {
        "timestamp": chat_session["start_time"],
        "model": chat_session["model"],
        "model_specs": MODELS[chat_session["model"]],
        "messages": chat_session["messages"]
    }

    with open(filename, 'w') as f:
        json.dump(transcript_data, f, indent=2)

    print(f"\nTranscript exported to {filename}")

def display_model_categories():
    """Display available model categories for selection."""
    categories = {
        "1": "GPT-4o Series",
        "2": "GPT-4o Mini Series",
        "3": "o1 Series",
        "4": "DALL-E Series",
        "5": "GPT-4o Realtime Preview",
        "6": "GPT-4o Audio Preview",
        "7": "GPT-4 Turbo and GPT-4",
        "8": "GPT-3.5 Turbo"
    }

    print("\nAvailable Model Categories:")
    for num, category in categories.items():
        print(f"{num}. {category}")

    return categories

def display_models_in_category(category):
    """Display models within a selected category."""
    model_filters = {
        "1": {"prefix": "gpt-4o", "include": ["chatgpt-4o"]},
        "2": {"prefix": "gpt-4o-mini"},
        "3": {"prefix": "o1"},
        "4": {"prefix": "dall-e"},
        "5": {"prefix": "gpt-4o-realtime"},
        "6": {"prefix": "gpt-4o-audio"},
        "7": {"prefix": "gpt-4"},
        "8": {"prefix": "gpt-3.5"}
    }

    filter_info = model_filters[category]
    filtered_models = []

    # Filter models based on prefix and additional includes
    for model in MODELS.keys():
        if model.startswith(filter_info["prefix"]) or \
           ("include" in filter_info and any(inc in model for inc in filter_info["include"])):
            filtered_models.append(model)

    print("\nAvailable Models in Category:")
    for i, model in enumerate(filtered_models, 1):
        specs = MODELS[model]
        print(f"{i}. {model}")
        if "description" in specs:
            print(f"   Description: {specs['description']}")

    return filtered_models

def handle_dalle_request(prompt, model="dall-e-3", size="1024x1024", quality="standard", n=1):
    """Handle DALL-E image generation requests."""
    try:
        response = client.images.generate(
            model=model,
            prompt=prompt,
            size=size,
            quality=quality,
            n=n,
        )
        return {
            "urls": [item.url for item in response.data],
            "revised_prompt": response.data[0].revised_prompt if hasattr(response.data[0], 'revised_prompt') else None
        }
    except Exception as e:
        print(f"Error generating image: {str(e)}")
        return None

def send_message(chat_session, user_message):
    """Send a message to the API and return the response."""
    # Check if it's a DALL-E model
    if chat_session["model"].startswith("dall-e"):
        result = handle_dalle_request(user_message, model=chat_session["model"])
        if result:
            response_text = "Image generated successfully!\n"
            if result["revised_prompt"]:
                response_text += f"\nRevised prompt: {result['revised_prompt']}\n"
            response_text += "\nImage URLs:\n" + "\n".join(result["urls"])
        else:
            response_text = "Failed to generate image."

        # Add to conversation history
        chat_session["messages"].append({"role": "user", "content": user_message})
        chat_session["messages"].append({"role": "assistant", "content": response_text})
        return response_text

    # Regular chat models
    else:
        chat_session["messages"].append({"role": "user", "content": user_message})
        try:
            response = client.chat.completions.create(
                model=chat_session["model"],
                messages=chat_session["messages"],
                temperature=1.0
            )
            assistant_message = response.choices[0].message.content
            chat_session["messages"].append({"role": "assistant", "content": assistant_message})
            return assistant_message
        except Exception as e:
            print(f"Error: {str(e)}")
            return None

def initialize_chat(model="gpt-4o"):
    """Initialize a new chat session with specified model."""
    return {
        "model": model,
        "messages": [],
        "start_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

def start_interactive_chat():
    # Let user choose model category
    categories = display_model_categories()
    while True:
        category_choice = input("\nChoose category (number): ").strip()
        if category_choice in categories:
            break
        print("Invalid category. Please try again.")

    # Let user choose specific model
    filtered_models = display_models_in_category(category_choice)
    while True:
        model_num = input("\nChoose model number: ").strip()
        try:
            model_index = int(model_num) - 1
            if 0 <= model_index < len(filtered_models):
                model = filtered_models[model_index]
                break
        except ValueError:
            pass
        print("Invalid model number. Please try again.")

    # Initialize chat session
    chat_session = initialize_chat(model)
    print(f"\nStarting chat with {model}")
    model_info = MODELS[model]

    # Display model-specific instructions
    if model.startswith("dall-e"):
        print("\nDALL-E Image Generation Options:")
        if model == "dall-e-3":
            print("- Available sizes: 1024x1024, 1024x1792, 1792x1024")
            print("- Quality options: standard, hd")
            print("- One image per request")
            print("\nTip: To prevent prompt enhancement, start with:")
            print('"I NEED to test how the tool works with extremely simple prompts. DO NOT add any detail, just use it AS-IS:"')
        else:  # dall-e-2
            print("- Available size: 1024x1024")
            print("- Can generate up to 10 images per request")
    else:
        if "context_window" in model_info:
            print(f"Context Window: {model_info['context_window']} tokens")
            print(f"Max Output: {model_info['max_output']} tokens")
        if "description" in model_info:
            print(f"Description: {model_info['description']}")

    print("\nType 'exit' to end the conversation")
    print("Type 'transcript' to view the current transcript")

    # For DALL-E-3, add size and quality options
    if model == "dall-e-3":
        # Size selection
        sizes = {
            "1": "1024x1024",
            "2": "1024x1792",
            "3": "1792x1024"
        }
        print("\nAvailable Image Sizes:")
        for num, size in sizes.items():
            print(f"{num}. {size}")

        while True:
            size_choice = input("\nChoose size (number) [default: 1]: ").strip()
            if not size_choice:
                size_choice = "1"
            if size_choice in sizes:
                size = sizes[size_choice]
                break
            print("Invalid choice. Please try again.")

        # Quality selection
        qualities = {
            "1": "standard",
            "2": "hd"
        }
        print("\nAvailable Quality Options:")
        for num, qual in qualities.items():
            print(f"{num}. {qual}")

        while True:
            quality_choice = input("\nChoose quality (number) [default: 1]: ").strip()
            if not quality_choice:
                quality_choice = "1"
            if quality_choice in qualities:
                quality = qualities[quality_choice]
                break
            print("Invalid choice. Please try again.")

        chat_session["dalle_options"] = {"size": size, "quality": quality}

    print("-" * 80)

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'transcript':
            print_transcript(chat_session)
            continue

        if user_input:
            response = send_message(chat_session, user_input)
            if response:
                print("\nAssistant:", response)

    # Print final transcript
    print_transcript(chat_session)

    # Ask if user wants to export
    if input("\nExport transcript to file? (y/n): ").lower() == 'y':
        export_transcript(chat_session)

# **3. Start the interactive chat with a model of your choice**
Finally, run this cell to actually start the conversation with a model of your choice. You will first be asked to pick a 'model family (e.g. 4o family, o1 family)' and then the actual model you wish to chat with by typing in a number. Type 'exit' if you wish to end the conversation. You will be given the option to export the conversation transcript in .json at the end.

  See the full list of models that OpenAI provides here for more information about the models: https://platform.openai.com/docs/models.

In [28]:
# Run the interactive chat
start_interactive_chat()


Available Model Categories:
1. GPT-4o Series
2. GPT-4o Mini Series
3. o1 Series
4. DALL-E Series
5. GPT-4o Realtime Preview
6. GPT-4o Audio Preview
7. GPT-4 Turbo and GPT-4
8. GPT-3.5 Turbo

Choose category (number): 4

Available Models in Category:
1. dall-e-3
   Description: Latest DALL·E model released in Nov 2023
2. dall-e-2
   Description: Previous DALL·E model with 4x greater resolution than original

Choose model number: 1

Starting chat with dall-e-3

DALL-E Image Generation Options:
- Available sizes: 1024x1024, 1024x1792, 1792x1024
- Quality options: standard, hd
- One image per request

Tip: To prevent prompt enhancement, start with:
"I NEED to test how the tool works with extremely simple prompts. DO NOT add any detail, just use it AS-IS:"

Type 'exit' to end the conversation
Type 'transcript' to view the current transcript

Available Image Sizes:
1. 1024x1024
2. 1024x1792
3. 1792x1024

Choose size (number) [default: 1]: 1

Available Quality Options:
1. standard
2. hd

Cho